<a href="https://colab.research.google.com/github/kcalizadeh/phil_nlp/blob/master/w2v.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports and Mounting Drive

In [15]:
# this cell mounts drive, sets the correct directory, then imports all functions
# and relevant libraries via the functions.py file
from google.colab import drive
import sys

# install relevent libraries not included with colab
!pip install lime
!pip install symspellpy

drive.mount('/gdrive',force_remount=True)

drive_path = '/gdrive/MyDrive/Colab_Projects/Phil_NLP'

sys.path.append(drive_path)

Mounted at /gdrive


In [16]:
from functions import *
%load_ext autoreload
%autoreload 2

np.random_seed=17

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load the Data

In [17]:
df = pd.read_csv('/gdrive/MyDrive/Colab_Projects/Phil_NLP/phil_nlp.csv')

df.sample(5)

,title,author,school,sentence_spacy,sentence_str,sentence_length,sentence_lowered,lemmatized_str,tokenized_txt
53117,complete works vol 1,Aristotle,aristotle,"what the thinking power is, or the perceptive,...","what the thinking power is, or the perceptive,...",211,"what the thinking power is, or the perceptive,...","what the thinking power be , or the perceptiv...","['what', 'the', 'thinking', 'power', 'is', ','..."
123397,search after truth,Malebranche,rationalism,and His immutable will.,and His immutable will.,23,and his immutable will.,and -PRON- immutable will .,"['and', 'His', 'immutable', 'will', '.']"
318166,political economy,Ricardo,capitalism,An extraordinary stimulus may be also given fo...,An extraordinary stimulus may be also given fo...,388,an extraordinary stimulus may be also given fo...,an extraordinary stimulus may be also give fo...,"['An', 'extraordinary', 'stimulus', 'may', 'be..."
95555,essay concerning human understanding bk 2,Locke,empiricism,Contrary testimonies.,Contrary testimonies.,21,contrary testimonies.,contrary testimony .,"['Contrary', 'testimonies', '.']"
118901,search after truth,Malebranche,rationalism,It must therefore be concluded from the experi...,It must therefore be concluded from the experi...,372,it must therefore be concluded from the experi...,-PRON- must therefore be conclude from the ex...,"['It', 'must', 'therefore', 'be', 'concluded',..."


In [18]:
# using gensim's built-in tokenizer 
df['gensim_tokenized'] = df['sentence_str'].map(lambda x: simple_preprocess(x.lower(),deacc=True,
                                                        max_len=100))

In [19]:
# check how it worked
print(df.iloc[216066]['sentence_str'])
df['gensim_tokenized'][216066]

The bumble bee is a part of the reproductive system of the clover.


['the',
 'bumble',
 'bee',
 'is',
 'part',
 'of',
 'the',
 'reproductive',
 'system',
 'of',
 'the',
 'clover']

Well with that beautiful little quote, we are ready to start training our w2v model! At first we'll focus on a single school, since a single school is more likely to have consistency in their use of a word.

Unfortunately, we didn't have much luck with just training on the texts alone. The code for it is left here for posterity, but it was when we worked with GloVe as the base that we had results that were actually useful.

### Word 2 Vec Training

#### German Idealism as a Test Case

We start by examining the texts of German Idealism to get a feel for what kind of parameters would work best.

In [20]:
def make_w2v(series, stopwords=[], size=200, window=5, min_count=5, workers=-1, 
             epochs=20, lowercase=True, sg=0, seed=17, cbow_mean=1, alpha=0.025,
             sample=0.001, use_bigrams=True, threshold=10, bigram_min=5):
  # turn the series into a list, lower it, clean it
    sentences = [sentence for sentence in series]
    if lowercase:
      cleaned = []
      for sentence in sentences:
        cleaned_sentence = [word.lower() for word in sentence]
        cleaned_sentence = [word for word in sentence if word not in stopwords]
        cleaned.append(cleaned_sentence)
    else:
      cleaned = []
      for sentence in sentences:
        cleaned_sentence = [word for word in sentence]
        cleaned_sentence = [word for word in sentence if word not in stopwords]
        cleaned.append(cleaned_sentence)

  # incorporate bigrams
    if use_bigrams:
      bigram = Phrases(cleaned, min_count=bigram_min, threshold=threshold, delimiter=b' ')
      bigram_phraser = Phraser(bigram)
      tokens_list = []
      for sent in cleaned:
        tokens_ = bigram_phraser[sent]
        tokens_list.append(tokens_)
      cleaned = tokens_list
    else:
      cleaned = cleaned

  # build the model
    model = Word2Vec(cleaned, size=size, window=window, 
                     min_count=min_count, workers=workers, seed=seed, sg=sg,
                     cbow_mean=cbow_mean, alpha=alpha, sample=sample)
    model.train(series, total_examples=model.corpus_count, epochs=epochs)
    model_wv = model.wv
    
  # clear it to avoid unwanted transference
    del model

    return model_wv

In [21]:
gi_wv = make_w2v(df[df['school'] == 'german_idealism']['gensim_tokenized'], threshold=12)

We can check this model by trying out a few words. For that purpose we have a testing function that tries some common word combinations.

In [22]:
pairs_to_try = [(['law', 'moral'], []),
                (['self', 'consciousness'], []),
                (['dialectic'], []),
                (['logic'], []),
]

In [23]:
test_w2v_pos_neg(gi_wv, pairs_to_try)

Positive - ['law', 'moral']	Negative - []
- aesthetically (0.24603)
- bodily (0.24569)
- inverse ratio (0.23537)
- new (0.22829)
- synonymous (0.22511)

Positive - ['self', 'consciousness']	Negative - []
- corresponding (0.25857)
- que (0.25134)
- patriarchal (0.24172)
- rhythm (0.22662)
- objecta (0.2262)

Positive - ['dialectic']	Negative - []
- situations (0.24978)
- intelligible substrate (0.24178)
- give rise (0.24007)
- endlessly (0.23979)
- indianapolis ind (0.22153)

Positive - ['logic']	Negative - []
- vaguely (0.2435)
- events (0.23107)
- projects (0.22523)
- religion (0.22428)
- indepen (0.22402)



Although some of these make a modicum of sense a lot of them seem like just gibberish. Let's try messing with some parameters.



##### Trying Skip-gram instead of C-bow

In [24]:
# make a base model with the preset parameters
skip_gi_wv = make_w2v(series = df[df['school'] == 'german_idealism']['gensim_tokenized'], 
                         stopwords=[], sg=1, seed=0)

In [25]:
test_w2v(skip_gi_wv, pairs_to_try)

Positive - ['law', 'moral']	Negative - []
- mistakes (0.21409)
- do (0.2132)
- remaining (0.21286)
- ingredient (0.21154)
- unceasing (0.21109)

Positive - ['self', 'consciousness']	Negative - []
- ultimate decision (0.28703)
- perfected (0.26523)
- despite (0.24827)
- loc (0.22929)
- education (0.22864)

Positive - ['dialectic']	Negative - []
- transcendental use (0.28578)
- indivisible (0.24304)
- communicate (0.23995)
- unconscious (0.23945)
- ethi (0.22595)

Positive - ['logic']	Negative - []
- playing (0.28748)
- continued (0.27647)
- intentions (0.24647)
- substratum (0.23162)
- fettered (0.23124)



These seem mildy more sensible. Let's tweak the other parameters.

##### Parameter Testing

In [26]:
model_v1 = make_w2v(df[df['school'] == 'german_idealism']['gensim_tokenized'],
                       stopwords=[],
                       size=500,
                       window=5,
                       min_count=25,
                       epochs=10,
                       sg=1, 
                       seed=45)

len(model_v1.vocab)

2928

In [27]:
test_w2v(model_v1, pairs_to_try)

Positive - ['law', 'moral']	Negative - []
- positively (0.154)
- association (0.14129)
- demand (0.13406)
- toward (0.12594)
- legislative (0.11773)

Positive - ['self', 'consciousness']	Negative - []
- absence (0.1558)
- natural science (0.15302)
- wants (0.13851)
- instinct (0.13707)
- commands (0.13615)

Positive - ['dialectic']	Negative - []
- assuming (0.18621)
- inwardly (0.16957)
- sublating (0.15569)
- disappears (0.14731)
- comprehension (0.14721)

Positive - ['logic']	Negative - []
- emerge (0.14531)
- range (0.14274)
- quantum (0.13392)
- outer (0.13161)
- clearly (0.12611)



Despite tweaking parameters far and wide, it's difficult to get any results that are compellingly sensible. In most cases there are one or two terms in the similarity list that make some sense but others that are just strange or unconnected

#### Trying Another School

In [28]:
cm_w2v = make_w2v(df[df['school'] == 'communism']['gensim_tokenized'],
                       stopwords=[],
                       size=700,
                       window=10,
                       min_count=10,
                       epochs=25,
                       sg=1, 
                       seed=10)

type(cm_w2v)

gensim.models.keyedvectors.Word2VecKeyedVectors

In [29]:
pairs_to_try=[(['material', 'conditions'], []),
              (['worker'], ['owner']),
              (['alienation', 'labor'], []),
              (['capital'], [])]

In [30]:
test_w2v(cm_w2v, pairs_to_try)

Positive - ['material', 'conditions']	Negative - []
- export (0.12331)
- points (0.11569)
- consumption (0.1128)
- man (0.11206)
- re appears (0.11086)

Positive - ['worker']	Negative - ['owner']
- bernsteinism (0.11703)
- accumulation (0.11645)
- processes (0.11398)
- brings about (0.1123)
- agitators (0.1074)

Positive - ['alienation', 'labor']	Negative - []
- succession (0.13823)
- clearly (0.13428)
- intellectual (0.12619)
- advantage (0.12468)
- fairly (0.12392)

Positive - ['capital']	Negative - []
- variable part (0.1325)
- draft (0.12887)
- historical (0.12735)
- seem (0.12625)
- industrial reserve (0.11934)



Here the results were similar - a few words that made some sense and plenty that were just odd.  

### Transfer Learning with GloVe

We'll import GloVe vectors as w2v, then use those as a base from which to train new vectors that are tuned to our corpus.

In [31]:
glove_file = datapath('/gdrive/MyDrive/Colab_Projects/Phil_NLP/glove.6B.50d.txt')
tmp_file = get_tmpfile("test_word2vec.txt")

_ = glove2word2vec(glove_file, tmp_file)

glove_vectors = KeyedVectors.load_word2vec_format(tmp_file)

In [45]:
pairs_to_try = [(['law', 'moral'], []),
                (['self', 'consciousness'], []),
                (['dialectic'], []),
                (['logic'], []),
]

In [33]:
# check out how GloVe works on our test pairs
test_w2v_pos_neg(glove_vectors, pairs_to_try)

Positive - ['law', 'moral']	Negative - []
- morality (0.82654)
- legal (0.82652)
- laws (0.81529)
- constitutional (0.80616)
- fundamental (0.80217)

Positive - ['self', 'consciousness']	Negative - []
- sense (0.83446)
- mind (0.79755)
- vision (0.78202)
- belief (0.78031)
- life (0.77984)

Positive - ['dialectic']	Negative - []
- hegelian (0.88376)
- dialectical (0.83417)
- dialectics (0.80672)
- materialist (0.77674)
- metaphysics (0.77488)

Positive - ['logic']	Negative - []
- reasoning (0.81405)
- intuitionistic (0.76531)
- concepts (0.75831)
- logical (0.75604)
- theory (0.75026)



Ok these make a lot more sense right from the start. But we want them to be trained on our actual philosophical texts - that way we can see how different thinkers use different words and potentially use the vectors for classification.

So in the cells below we train the existing GloVe model on on the German Idealist texts as a test.

In [40]:
# isolate the relevant school
documents = df[df['school'] == 'german_idealism']['gensim_tokenized']

# format the series to be used
stopwords = []

sentences = [sentence for sentence in documents]
cleaned = []
for sentence in sentences:
  cleaned_sentence = [word.lower() for word in sentence]
  cleaned_sentence = [word for word in sentence if word not in stopwords]
  cleaned.append(cleaned_sentence)

# get bigrams
bigram = Phrases(cleaned, min_count=20, threshold=10, delimiter=b' ')
bigram_phraser = Phraser(bigram)

bigramed_tokens = []
for sent in cleaned:
    tokens = bigram_phraser[sent]
    bigramed_tokens.append(tokens)

# run again to get trigrams
trigram = Phrases(bigramed_tokens, min_count=20, threshold=10, delimiter=b' ')
trigram_phraser = Phraser(trigram)

trigramed_tokens = []
for sent in bigramed_tokens:
    tokens = trigram_phraser[sent]
    trigramed_tokens.append(tokens)

# build a toy model to update with
base_model = Word2Vec(size=300, min_count=5)
base_model.build_vocab(trigramed_tokens)
total_examples = base_model.corpus_count

# add GloVe's vocabulary & weights
base_model.build_vocab([list(glove_vectors.vocab.keys())], update=True)

# train on our data
base_model.train(trigramed_tokens, total_examples=total_examples, epochs=base_model.epochs)
base_model_wv = base_model.wv
del base_model

In [46]:
test_w2v(base_model_wv, pairs_to_try)

Positive - ['law', 'moral']	Negative - []
- freedom (0.82361)
- morality (0.79375)
- rule (0.78852)
- moral law (0.78502)
- happiness (0.78457)

Positive - ['self', 'consciousness']	Negative - []
- self consciousness (0.92599)
- essence (0.87534)
- objectivity (0.85972)
- externality (0.85939)
- immediacy (0.85824)

Positive - ['dialectic']	Negative - []
- antinomy (0.89444)
- doctrine (0.89214)
- remainder (0.88909)
- resolution (0.87126)
- procedure (0.86174)

Positive - ['logic']	Negative - []
- doctrine (0.84756)
- pure reason (0.84016)
- science (0.82024)
- metaphysics (0.81265)
- idealism (0.80801)



We can immediately see that these make a lot more sense (and the similarity scores are a lot higher). 'Self' + 'consciousness' is rightly associated with 'self consciousness' and 'moral' + 'law' with 'moral law'. It even identifies the German Idealist tendency to unify logic and metaphysics. 

This is a massive improvement - these vectors can be fairly said to reflect how german idealists use these terms. Moreover, they are significantly different than the original GloVe model, which indicates that there was real learning going on here.

For comparison, let's check these same terms, but as used by Phenomenologists.

In [42]:
def train_glove(school, glove_vectors, threshold=10, stopwords=[],
                min_count=20):
  # isolate the relevant school
  documents = df[df['school'] ==school]['gensim_tokenized']

  # format the series to be used
  stopwords = []

  sentences = [sentence for sentence in documents]
  cleaned = []
  for sentence in sentences:
    cleaned_sentence = [word.lower() for word in sentence]
    cleaned_sentence = [word for word in sentence if word not in stopwords]
    cleaned.append(cleaned_sentence)

  # get bigrams
  bigram = Phrases(cleaned, min_count=min_count, threshold=threshold, 
                   delimiter=b' ')
  bigram_phraser = Phraser(bigram)

  bigramed_tokens = []
  for sent in cleaned:
      tokens = bigram_phraser[sent]
      bigramed_tokens.append(tokens)

  # run again to get trigrams
  trigram = Phrases(bigramed_tokens, min_count=min_count, threshold=threshold, 
                    delimiter=b' ')
  trigram_phraser = Phraser(trigram)

  trigramed_tokens = []
  for sent in bigramed_tokens:
      tokens = trigram_phraser[sent]
      trigramed_tokens.append(tokens)

  # build a toy model to update with
  model = Word2Vec(size=300, min_count=5)
  model.build_vocab(trigramed_tokens)
  total_examples = model.corpus_count

  # add GloVe's vocabulary & weights
  model.build_vocab([list(glove_vectors.vocab.keys())], update=True)

  # train on our data
  model.train(trigramed_tokens, total_examples=total_examples, epochs=model.epochs)
  model_wv = model.wv
  del model
  return model_wv

In [43]:
pairs_to_try = [(['perception'], []),
                (['dasein'], []),
                (['consciousness'], []),
                (['method'], []),]

In [44]:
ph_model = train_glove(school='phenomenology', glove_vectors=glove_vectors)

test_w2v(ph_model, pairs_to_try)

Positive - ['perception']	Negative - []
- consciousness (0.92737)
- care (0.92598)
- reality (0.92137)
- movement (0.91659)
- representation (0.914)

Positive - ['dasein']	Negative - []
- being (0.89199)
- itself (0.86746)
- truth (0.85501)
- understanding (0.82309)
- everything (0.81982)

Positive - ['consciousness']	Negative - []
- perception (0.92737)
- care (0.91024)
- movement (0.8974)
- knowledge (0.8942)
- reality (0.88805)

Positive - ['method']	Negative - []
- necessity (0.95785)
- spirit (0.95634)
- origin (0.95564)
- historicity (0.95134)
- condition (0.95012)



Using the phenomenology vectors on some central terms of phenomenology once again yields some pretty compelling results. 

These vectors seem to be an effective tool for revealing how a word is used by a school. 

As a final kind of exploration of this method, we'll train w2v models in this way for each school and examine how each of them looks a couple of the same words. 

In [58]:
w2v_dict = {}

for school in df['school'].unique():
  w2v_dict[school] = train_glove(school, glove_vectors=glove_vectors)
  print(f'{school} completed')

plato completed
aristotle completed
empiricism completed
rationalism completed
analytic completed
continental completed
phenomenology completed
german_idealism completed
communism completed
capitalism completed


In [59]:
for school in df['school'].unique():
  print(f'\t{school.upper()}')
  print('----------------------')
  test_w2v(w2v_dict[school], [(['philosophy'], [])])

	PLATO
----------------------
Positive - ['philosophy']	Negative - []
- relief (0.94646)
- springs (0.94496)
- friendship (0.94237)
- greece (0.93422)
- regime (0.93311)

	ARISTOTLE
----------------------
Positive - ['philosophy']	Negative - []
- respiration (0.91699)
- oratory (0.87146)
- shrillness (0.87085)
- holders (0.86975)
- memory (0.85944)

	EMPIRICISM
----------------------
Positive - ['philosophy']	Negative - []
- religion (0.93495)
- mankind (0.92261)
- doctrine (0.92259)
- inquiry (0.9155)
- faith (0.90897)

	RATIONALISM
----------------------
Positive - ['philosophy']	Negative - []
- person (0.95782)
- return (0.92869)
- death (0.92862)
- fall (0.92596)
- public (0.9258)

	ANALYTIC
----------------------
Positive - ['philosophy']	Negative - []
- philosophical (0.90815)
- hahn (0.85215)
- carnap (0.84692)
- semantics (0.84195)
- davidson (0.83312)

	CONTINENTAL
----------------------
Positive - ['philosophy']	Negative - []
- metaphysics (0.9689)
- unreason (0.95789)
- hist

Interestingly, many of these top words align quite strongly with the school's general attitude towards philosophy. 

The model seems solid - our next step is to train one on the entire corpus for use in classification. We do that, and export it, below.

In [51]:
documents = df['gensim_tokenized']

# format the series to be used
stopwords = []

sentences = [sentence for sentence in documents]
cleaned = []
for sentence in sentences:
  cleaned_sentence = [word.lower() for word in sentence]
  cleaned_sentence = [word for word in sentence if word not in stopwords]
  cleaned.append(cleaned_sentence)

# get bigrams
bigram = Phrases(cleaned, min_count=30, threshold=10, 
                  delimiter=b' ')
bigram_phraser = Phraser(bigram)

bigramed_tokens = []
for sent in cleaned:
    tokens = bigram_phraser[sent]
    bigramed_tokens.append(tokens)

# run again to get trigrams
trigram = Phrases(bigramed_tokens, min_count=30, threshold=10, 
                  delimiter=b' ')
trigram_phraser = Phraser(trigram)

trigramed_tokens = []
for sent in bigramed_tokens:
    tokens = trigram_phraser[sent]
    trigramed_tokens.append(tokens)

# build a toy model to update with
all_text_model = Word2Vec(size=300, min_count=5)
all_text_model.build_vocab(trigramed_tokens)
total_examples = all_text_model.corpus_count

# add GloVe's vocabulary & weights
all_text_model.build_vocab([list(glove_vectors.vocab.keys())], update=True)

# train on our data
all_text_model.train(trigramed_tokens, total_examples=total_examples, 
                     epochs=all_text_model.epochs)
all_text_wv = all_text_model.wv


As a test case, let's see how the philosophy thinks of itself as compared to how glove thinks of philosophy.

In [52]:
for model in [1, 2]:
  if model == 1:
    print(f'\tPHILOSOPHY CORPUS')
    print('------------------------------------')
    test_w2v(all_text_wv, [(['philosophy'], [])])
  if model == 2:
    print(f'\tBASE GLOVE')
    print('------------------------------------')
    test_w2v(glove_vectors, [(['philosophy'], [])])


	PHILOSOPHY CORPUS
------------------------------------
Positive - ['philosophy']	Negative - []
- metaphysics (0.78942)
- theology (0.76947)
- religion (0.73203)
- science (0.72824)
- philosophical (0.71864)

	BASE GLOVE
------------------------------------
Positive - ['philosophy']	Negative - []
- theology (0.88151)
- philosophical (0.84362)
- mathematics (0.83389)
- psychology (0.82387)
- sociology (0.81085)



This sort of stands to reason - 'metaphysics' often has a different meaning outside of philosophical discussion, so it's not surprising to see it as the most changed term here. 

All in all, things look good, so let's export the vectors so that they can be used in our neural networks. 

In [57]:
all_text_wv.save('/gdrive/MyDrive/Colab_Projects/Phil_NLP/w2v_models/w2v_for_nn.bin')
all_text_wv.save('/gdrive/MyDrive/Colab_Projects/Phil_NLP/w2v_models/w2v_for_nn.wordvectors')

In [60]:
for school in w2v_dict.keys():
  w2v_dict[school].save(f'/gdrive/MyDrive/Colab_Projects/Phil_NLP/w2v_models/{school}_w2v.wordvectors')

And that's it! See our other notebooks for more of the modeling work. 